In [24]:
import numpy as np
import pandas as pd

rs = np.random.RandomState(np.random.MT19937(np.random.SeedSequence(42)))

# Input
INTERNAL_RADIUS = 5.0
EXTERNAL_RADIUS = 6.5
M_1_RADIUS = 20 # 5
N_DEGREES = 4 # 4
ISOTERMA = 500
TEMP_INT = np.array([1500 for i in range(N_DEGREES)])
TEMP_EXT = np.random.normal(loc=150, scale=10, size=N_DEGREES)
RADIUSES = np.linspace(INTERNAL_RADIUS, EXTERNAL_RADIUS, M_1_RADIUS+1)
DEGREES = np.linspace(0.0, 2*np.pi, N_DEGREES)

delta_r = (EXTERNAL_RADIUS - INTERNAL_RADIUS)/(M_1_RADIUS)
delta_g = 2 * np.pi / N_DEGREES
size = (M_1_RADIUS) * (N_DEGREES)

5 6 5 4 500 1
1500 1500 1500 1500 160.8435309947052 162.42036793522266 159.59264334173378 144.6919884216488 

In [25]:
delta_r

0.075

In [26]:
radiuses = [INTERNAL_RADIUS + i * delta_r for i in range(M_1_RADIUS+1)]

In [27]:
RADIUSES

array([5.   , 5.075, 5.15 , 5.225, 5.3  , 5.375, 5.45 , 5.525, 5.6  ,
       5.675, 5.75 , 5.825, 5.9  , 5.975, 6.05 , 6.125, 6.2  , 6.275,
       6.35 , 6.425, 6.5  ])

In [28]:
radiuses

[5.0,
 5.075,
 5.15,
 5.225,
 5.3,
 5.375,
 5.45,
 5.525,
 5.6,
 5.675,
 5.75,
 5.825,
 5.9,
 5.975,
 6.05,
 6.125,
 6.2,
 6.275,
 6.35,
 6.425,
 6.5]

In [3]:
A = np.zeros((size, size))

A.shape

(20, 20)

$$ A \in \mathcal{R}^{(m+1 * n) \times (m+1 * n)} $$

La ecuacion que en el estacionario dice que

\begin{equation} 
   \frac{\partial^2 T}{\partial r^2} (r, \theta) + \frac{\partial T}{r \partial r}(r, \theta) + \frac{\partial^2 T}{r^2 \partial \theta^2}(r, \theta) = 0
\end{equation}

Usando diferencias finitas tenemos que

\begin{equation} 
   \frac{t_{j-1,k} - 2 t_{j,k} + t_{j+1,k}}{(\Delta r)^2} + \frac{t_{j,k} - t_{j-1,k}}{r \Delta r} + \frac{t_{j,k-1} - 2 t_{j,k} + t_{j,k+1}}{r^2(\Delta \theta)^2} = 0
\end{equation}

Distribuyendo para obtener los coeficientes de cada variable obtenemos

1. $t_{j-1,k} = (\Delta r)^{-2} - (r \Delta r)^{-1}$
2. $t_{j , k} = (r \Delta r)^{-1} - 2(\Delta r)^{-2} - 2(r^2(\Delta\theta)^2)^{-1}$
3. $t_{j+1,k} = (\Delta r)^{-2}$
4. $t_{j,k-1} = (r^2 (\Delta\theta)^2)^{-1}$
5. $t_{j,k+1} = (r^2 (\Delta\theta)^2)^{-1}$

En total hay $(m+1) n$ puntos donde deberemos calcular la temperatura

Entonces, tenemos el vector $x=(t_{0,0},\dots,t_{0,n},t_{1,0},\dots,t_{m+1,0},\dots,t_{m+1,n})$. Si definimos la matriz $A$ de forma tal que al hacer $Ax=b$, resolvemos el sistema. Los $t_{0,k}$ y $t_{m+1,k}$ son conocidos, con $k=1,\dots,n$


Como $r, \theta, \Delta r, \Delta \theta$ son datos, los coeficientes son todos numericos ($r$ es el valor del radio en cada punto y $\theta$ el angulo, pero no son incognitas, dependen del $j,k$)

In [4]:
for i in range(1, M_1_RADIUS-1):
    for j in range(N_DEGREES):
        r = RADIUSES[i]
        # print(f"{i}:{r} - {j}:{DEGREES[j]}")
        v = np.zeros((M_1_RADIUS,N_DEGREES))
        
        v[i-1][j] = (delta_r) ** -2 - (r * delta_r) ** -1
        v[i][j] = (r * delta_r) ** -1 - 2 * (delta_r)**-2 - 2 * (r**2 * delta_g**2)**-1
        v[i+1][j] = delta_r ** -2
        tmp = (r**2 * delta_g**2) ** -1
        
        v[i][(j-1) % N_DEGREES] = tmp
        v[i][(j+1) % N_DEGREES] = tmp

        A[i*N_DEGREES+j] = v.reshape(size)

In [5]:
first = np.array([-A[N_DEGREES][0] * 1500 for i in range(N_DEGREES)])
end = np.array([-delta_r**-2 * TEMP_EXT[i] for i in range(N_DEGREES)])
b = np.concatenate([first, np.zeros(A.shape[0] - 4 * N_DEGREES), end])

In [6]:
b.shape

(12,)

In [7]:
total = len(A)
App = pd.DataFrame(A).loc[N_DEGREES:total-N_DEGREES-1, N_DEGREES:total-N_DEGREES-1].to_numpy()

In [8]:
App.shape

(12, 12)

In [9]:
x = np.linalg.solve(App, b)

In [10]:
x

array([1143.92005744, 1142.56459792, 1142.90319533, 1140.97999607,
        801.5380726 ,  798.87569567,  799.54223917,  795.76553815,
        471.84159137,  467.91416131,  468.90082567,  463.33149203])

In [11]:
np.linalg.cond(App)

5.836867206714859